# Part 1 — Polars Eager Pipeline

This notebook is the **Polars (eager mode)** version of the invoice KPI pipeline. It replicates the same logic as the original pandas script (`refined_transformation_indicators.py`) but uses **Polars** instead.

**Key concepts demonstrated:**
- `pl.read_database_uri()` for reading SQL queries
- `pl.when / then / otherwise` for conditional classification
- `.join()`, `.group_by()`, `.agg()` for transformations
- `pl.concat()` for combining DataFrames
- Performance tracking with `time` and `tracemalloc`

In [ ]:
import time
import tracemalloc
import yaml
import polars as pl
from datetime import date
from pathlib import Path
from sqlalchemy import create_engine

from utils import get_db_engine, load_sql_file

# Start performance tracking
tracemalloc.start()
start_time = time.time()

# Configuration
BASE_DIR = Path.cwd()
CREDS_PATH = Path("C:/Users/roger.lloret/Documents/creds/creds_dmbi.yml")
SQL_DIR = BASE_DIR / "sql"
TABLE_NAME = "gen_kpi_ft"

## Helper Functions

Define Polars-specific helpers for database I/O and invoice classification.

| Function | Purpose |
|----------|---------|
| `read_from_database_pl` | Uses `pl.read_database_uri()` with a full connection URI |
| `write_to_database_pl` | Converts to pandas for MySQL writes via `to_sql()` |
| `classify_invoices_pl` | Uses `pl.when/then/otherwise` instead of `np.select` |

In [ ]:
def read_from_database_pl(engine, query: str) -> pl.DataFrame:
    """Reads a SQL query into a Polars DataFrame via the SQLAlchemy engine."""
    # render_as_string with hide_password=False to include credentials in the URI
    uri = engine.url.render_as_string(hide_password=False)
    return pl.read_database_uri(query, uri=uri)


def write_to_database_pl(engine, df: pl.DataFrame, table_name: str, if_exists: str = "append"):
    """Writes a Polars DataFrame to MySQL by converting to pandas for to_sql."""
    with engine.begin() as connection:
        df.to_pandas().to_sql(table_name, con=connection, if_exists=if_exists, index=False)


def classify_invoices_pl(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    """Classifies numeric values into categories using Polars when/then/otherwise."""
    return df.with_columns(
        pl.when(pl.col(column_name) < 50)
        .then(pl.lit("Less than 50"))
        .when((pl.col(column_name) >= 50) & (pl.col(column_name) <= 100))
        .then(pl.lit("Between 50 and 100"))
        .when(pl.col(column_name) > 100)
        .then(pl.lit("Greater than 100"))
        .otherwise(pl.lit("Invalid Input"))
        .alias("category")
    )

## 1. Load Credentials & Initialize Engine

In [ ]:
with open(CREDS_PATH, "r") as file:
    creds = yaml.safe_load(file)

db_engine = get_db_engine(creds["data_warehouse"])
print("Database engine created.")

## 2. Extract Data
Load invoices and contracts using SQL files and `pl.read_database_uri()`.

In [ ]:
invoices_sql = load_sql_file(SQL_DIR / "invoices_main.sql")
contracts_sql = load_sql_file(SQL_DIR / "contracts_main.sql")

invoices_df = read_from_database_pl(db_engine, invoices_sql)
contracts_df = read_from_database_pl(db_engine, contracts_sql)

print(f"Invoices: {invoices_df.shape[0]} rows, {invoices_df.shape[1]} columns")
print(f"Contracts: {contracts_df.shape[0]} rows, {contracts_df.shape[1]} columns")
invoices_df.head()

## 3. Transform Data
- Deduplicate contracts
- Classify invoices using `pl.when / then / otherwise`
- Cast join keys to `Int64`
- Left join invoices with contracts

In [ ]:
# Deduplicate contracts
contracts_df = contracts_df.unique()

# Classify invoices
invoices_df = classify_invoices_pl(invoices_df, "total_import_euros")

# Cast join keys to numeric (Int64)
invoices_df = invoices_df.with_columns(pl.col("contract_id").cast(pl.Int64, strict=False))
contracts_df = contracts_df.with_columns(pl.col("contract_id").cast(pl.Int64, strict=False))

# Merge
merged_df = invoices_df.join(contracts_df, on="contract_id", how="left")
print(f"Merged: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
merged_df.head()

## 4. Create KPIs
- **KPI 1:** Total amount by category and client type
- **KPI 2:** Invoice count by document type

In [ ]:
# KPI 1: Amount by Category & Client Type
kpi_category_df = (
    merged_df
    .group_by(["category", "client_type_description"])
    .agg(pl.col("total_import_euros").sum().alias("kpi_value"))
    .with_columns(
        (
            pl.lit("Total amount in euros of the customers with invoices ")
            + pl.col("category")
            + pl.lit(" euros and ")
            + pl.col("client_type_description")
        ).alias("kpi_name")
    )
    .select(["kpi_name", "kpi_value"])
)

# KPI 2: Document Type Count
kpi_doctype_df = (
    merged_df
    .group_by("document_type_description")
    .agg(pl.col("total_import_euros").count().cast(pl.Float64).alias("kpi_value"))
    .with_columns(
        (
            pl.lit("Number of invoices of invoice type ")
            + pl.col("document_type_description")
        ).alias("kpi_name")
    )
    .select(["kpi_name", "kpi_value"])
)

print("KPI 1 — Amount by Category:")
print(kpi_category_df)
print("\nKPI 2 — Document Type Count:")
print(kpi_doctype_df)

## 5. Aggregate and Load to Database

In [ ]:
main_df = pl.concat([kpi_category_df, kpi_doctype_df])
main_df = main_df.with_columns(pl.lit(date.today()).alias("kpi_date"))
output_df = main_df.select(["kpi_date", "kpi_name", "kpi_value"])

write_to_database_pl(db_engine, output_df, TABLE_NAME)
print("ETL Process completed successfully.")
output_df

## Performance Metrics

In [ ]:
elapsed = time.time() - start_time
_, peak_memory = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Polars (eager) pipeline executed in {elapsed:.2f} seconds.")
print(f"Peak RAM usage: {peak_memory / (1024 * 1024):.2f} MB")